# IEEE-CIS Fraud Detection Part 2

In this series of notebooks, we are working on a supervised, regression machine learning problem. Using Kaggle's competition [IEEE-CIS Fraud Detection](https://www.kaggle.com/c/ieee-fraud-detection) dataset, we want to predict whether a transaction is fraud or not. 

 ### Workflow 
 1. Understand the problem (we're almost there already)
 2. Exploratory Data Analysis
 3. Feature engineering to create a dataset for machine learning
 4. Create a baseline machine learning model
 5. Try more complex machine learning models
 6. Optimize the selected model
 7. Investigate model predictions in context of problem
 8. Draw conclusions and lay out next steps
 
The first notebook covered steps 1-3, and in this notebook, we will cover 4-6.

# Read in Data



In [1]:
# Numpy and pandas
import pandas as pd
import numpy as np

#Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno

# Statistics tools
import scipy.stats as stats

# Sklearn data clean
from sklearn.preprocessing import LabelEncoder
from dask_ml.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler

# Model selection

from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

# Logistic Regression
from dask_ml.linear_model import LogisticRegression

# KNN Classifer 
from sklearn.neighbors import KNeighborsClassifier

# Decision Trees
from sklearn import tree
from sklearn.tree import DecisionTreeRegressor
from IPython.display import Image
import pydotplus
import graphviz

# Random Forests 
from sklearn.ensemble import RandomForestClassifier

# SVM
from sklearn.svm import SVC

# Gradient Boost
from dask_ml.xgboost import XGBClassifier

# Evaluate
from sklearn import metrics
from sklearn.metrics import f1_score,roc_auc_score, confusion_matrix, classification_report, accuracy_score

# Hyperparameter tuning
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

# Datetime
from datetime import datetime

# Import data
import warnings

# Dask 
import dask.array as da
import dask.dataframe as dd
from dask.distributed import Client

from dask_ml.preprocessing import OneHotEncoder, MinMaxScaler
from dask_ml.datasets import make_classification
from dask_ml.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV



from dask_ml.wrappers import ParallelPostFit

from sklearn.externals.joblib import parallel_backend

In [2]:
# Load Data
df = dd.read_csv('/Users/tsawaengsri/Desktop/Data Science Courses/Datasets/ieee-fraud-detection/clean_train.csv')

In [3]:
# Display sizes of data
print('Training Feature Size: ', df.shape)

Training Feature Size:  (Delayed('int-64d0a26f-6565-4e1d-9c41-ddd6c6c0d905'), 1704)


In [4]:
df.head()

,TransactionID,isFraud,TransactionDT,TransactionAmt,card1,card2,card3,card5,addr1,addr2,...,browser_info_verykoolS5019,browser_info_verykoolS5524,browser_info_verykoolS5525,browser_info_verykoolS5530,browser_info_verykools4009,browser_info_verykools5004,browser_info_verykools5034,browser_info_verykools5035,browser_info_vivo,browser_info_xs-Z47b7VqTMxs
0,2987000,0,86400,68.5,13926,NaN,150.0,142.0,315.0,87.0,...,0,0,0,0,0,0,0,0,0,0
1,2987001,0,86401,29.0,2755,404.0,150.0,102.0,325.0,87.0,...,0,0,0,0,0,0,0,0,0,0
2,2987002,0,86469,59.0,4663,490.0,150.0,166.0,330.0,87.0,...,0,0,0,0,0,0,0,0,0,0
3,2987003,0,86499,50.0,18132,567.0,150.0,117.0,476.0,87.0,...,0,0,0,0,0,0,0,0,0,0
4,2987004,0,86506,50.0,4497,514.0,150.0,102.0,420.0,87.0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
df.info()

<class 'dask.dataframe.core.DataFrame'>
Columns: 1704 entries, TransactionID to browser_info_xs-Z47b7VqTMxs
dtypes: float64(402), int64(1302)

# Evaluating and Comparing Machine Learning Models
In this section we will build, train, and evalute several machine learning methods for our supervised regression task. The objective is to determine which model holds the most promise for further development (such as hyperparameter tuning).

## Split Train and Test set¶
Let's split dataset by using function train_test_split(). Here, the Dataset is broken into two parts in a ratio of 80:20. It means 80% data will be used for model training and 20% for model testing.

To continue feature selection, we will start by using the original attributes in the raw training set.

In [6]:
# Y is the target variable
y = df['isFraud']

# X is the feature set
X = df.drop(labels=['TransactionID','TransactionDT','isFraud'], axis=1).fillna(0)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [8]:
X_train = X_train.values
X_test = X_test.values
y_train = y_train.values
y_test = y_test.values

In [9]:
print('X_shapes:\n', 'X_train:', 'X_test:\n', X_train.shape, X_test.shape, '\n')
print('Y_shapes:\n', 'Y_train:', 'Y_test:\n', y_train.shape, y_test.shape)

X_shapes:
 X_train: X_test:
 (nan, 1701) (nan, 1701) 

Y_shapes:
 Y_train: Y_test:
 (nan,) (nan,)


### Imputing Missing Values 
Standard machine learning models cannot deal with missing values, and which means we have to find a way to fill these in or disard any features with missing values. Imputing also helps to reduce bias due to missingness: ‘rather than deleting cases that are subject to item-nonresponse, the sample size is maintained resulting in a potentially higher efficiency than for case deletion'[Durrant](https://www.tandfonline.com/doi/full/10.1080/1743727X.2014.979146#).

Here, we will fill in missing values with the mean of the column.

In [10]:
# Create an imputer object with a mean filling strategy
#imputer = SimpleImputer(strategy='mean')

# Train on the training features
#imputer.fit(X_train)

# Transform both training data and testing data
#X = imputer.transform(X_train)
#X_test = imputer.transform(X_test)

In [11]:
#print('Missing values in training features: ', np.sum(np.isnan(X)))
#print('Missing values in testing features:  ', np.sum(np.isnan(X_test)))

In [12]:
# Make sure all values are finite
#print(da.where(~da.isfinite(X)))
#print(da.where(~da.isfinite(X_test)))

After imputation, all of the features are real-valued.

### Scaling Features
The final step to take before we can build our models is to scale the features. This is necessary because features are in different units, and we want to normalize the features so the units do not affect the algorithm. Linear Regression and Random Forest do not require feature scaling, but other methods, such as support vector machines and k nearest neighbors, do require it because they take into account the Euclidean distance between observations. For this reason, it is a best practice to scale features when we are comparing multiple algorithms.

In [13]:
# Create the scaler object with a range of 0-1
scaler = MinMaxScaler(feature_range=(0, 1))

# Fit on the training data
scaler.fit(X)

# Transform both the training and testing data
X = scaler.transform(X)
X_test = scaler.transform(X_test)

In [14]:
# Convert y to one-dimensional array (vector)
y = da.array(y_train).reshape((-1, ))
y_test = da.array(y_test).reshape((-1, ))

# Baseline 
For a naive baseline, we will use logistic regression to predict the probability of fraud occurrence. Unlike linear regression which gives continuous output, logistic regression provides a constant output in prediciting binary classes. If the probability 'p' is greater than 0.5, the data is labeled '1'. Probability less than 0.5 is labeled as '0'.

## Logistic Regression Implementation
First, we'll create the model and train the model and make predictions on the testing data.

In [36]:
X, y = make_classification(chunks=10000)
X_test, y_test = make_classification(chunks=10000)

In [37]:
# Logisitic Regression 
start_time = datetime.now()

# Instantiate the model (using the default parameters)
logreg = LogisticRegression()

# Fit the model with data
logreg.fit(X, y)

# Predict on test set
y_pred=logreg.predict(X_test)

# Compute ROC AUC score, accuracy score, confusion matrix, and classification report
print('ROC AUC score: %0.4f' % roc_auc_score(y_test, y_pred))
print('Accuracy score: %0.4f' % accuracy_score(y_test, y_pred))      
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

end_time = datetime.now()
print('\nDuration: {}'.format(end_time - start_time))

ROC AUC score: 0.4467
Accuracy score: 0.4700
[[13 28]
 [25 34]]
              precision    recall  f1-score   support

           0       0.34      0.32      0.33        41
           1       0.55      0.58      0.56        59

   micro avg       0.47      0.47      0.47       100
   macro avg       0.45      0.45      0.45       100
weighted avg       0.46      0.47      0.47       100


Duration: 0:00:00.194093


As we can see, the ROC AUC and accuracy scores are very different. This model has an accuracy score of 96.5%. The score is calculated by adding the true positive and true negative values of 113,732 and 231 and dividing by the total count of 118,108. In contrast, the ROC AUC score is close to a random guess score of 52.7%. From the confusion matrix, we failed to flag 134 transactions as fraud out of 113,732 fraudulent transactions. 

In [38]:
# ROC Curve
y_pred_proba = logreg.predict_proba(X_test)[::,1]
fpr, tpr, _ = metrics.roc_curve(y_test,  y_pred_proba)
auc = metrics.roc_auc_score(y_test, y_pred_proba)
plt.plot(fpr,tpr,label="data 1, auc="+str(auc))
plt.legend(loc=4)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.show()

IndexError: Too many indices for array

The predicted probability of the logistic regression basline scored around 0.77 which is better than random guess score of 0.5.


## Improved Model: Random Forest
Let's try using a Random Forest on the same training data to see if it will beat the performance of our baseline. The Random Forest is a much more powerful model especially when we use hundreds of trees. We will use 100 trees in the random forest.

In [39]:
# Random Forest Classifer
start_time = datetime.now()

#Create a Gaussian Classifier
clf=RandomForestClassifier(n_estimators=100)

#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X, y)

y_pred=clf.predict(X_test)

# Compute ROC AUC score, accuracy score, confusion matrix, and classification report
print('ROC AUC score: %0.4f' % roc_auc_score(y_test, y_pred))
print('Accuracy score: %0.4f' % accuracy_score(y_test, y_pred))      
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

end_time = datetime.now()
print('\nDuration: {}'.format(end_time - start_time))

ROC AUC score: 0.4748
Accuracy score: 0.4900
[[16 25]
 [26 33]]
              precision    recall  f1-score   support

           0       0.38      0.39      0.39        41
           1       0.57      0.56      0.56        59

   micro avg       0.49      0.49      0.49       100
   macro avg       0.47      0.47      0.47       100
weighted avg       0.49      0.49      0.49       100


Duration: 0:00:00.387075


The ROC AUC and accuracy score for this model has increased from the baseline model indicating that random forest is a the better model for this data. 

In [ ]:
# ROC Curve
y_pred_proba = logreg.predict_proba(X_test)[::,1]
fpr, tpr, _ = metrics.roc_curve(y_test,  y_pred_proba)
auc = metrics.roc_auc_score(y_test, y_pred_proba)
plt.plot(fpr,tpr,label="data 1, auc="+str(auc))
plt.legend(loc=4)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.show()

This graph looks exactly like the baseline model. I'm not sure if this kernel ran properly. 

### Model Interpretation: Feature Importances
As a simple method to see which variables are the most relevant, we can look at the feature importances of the random forest. We may use these feature importances as a method of dimensionality reduction in future work.

In [40]:
feature_importances = dd.DataFrame(clf.feature_importances_,
                                   index = df.columns,
                                    columns=['importance']).sort_values('importance', ascending=False)

TypeError: __init__() got an unexpected keyword argument 'index'

In [41]:
#'Feature_importances_' from random forest
print((feature_importances).head(10))

NameError: name 'feature_importances' is not defined

## Models to Evaluate
We will compare five different machine learning models:

1. Logistic Regression
2. Random Forest Classifer
3. Support Vector Machine 
4. K-Nearest Neighbors Classifer
5. Extreme Gradient Boosting Classifer

In [44]:
# Function to calculate roc auc score
def auc(y_test, y_pred_proba):
    auc = metrics.roc_auc_score(y_test, y_pred_proba)
    return auc

# Takes in a model, trains the model, and evaluates the model on the test set
def fit_and_evaluate(model):
    
   # with joblib.parallel_backend('dask'):
        # Train the model
        model.fit(X, y)
    
        # Make predictions and evalute
        model_pred = model.predict(X_test)
    
        # Compute ROC AUC score, accuracy score, confusion matrix, and classification report
        print('ROC AUC score: %0.4f' % roc_auc_score(y_test, model_pred))
        print('Accuracy score: %0.4f' % accuracy_score(y_test, model_pred))      
        print(confusion_matrix(y_test, model_pred))
        print(classification_report(y_test, model_pred))
    
        # ROC Curve
        y_pred_proba = logreg.predict_proba(X_test)[::,1]
        fpr, tpr, _ = metrics.roc_curve(y_test,  y_pred_proba)
        auc = metrics.roc_auc_score(y_test, y_pred_proba)
        plt.plot(fpr,tpr,label="data 1, auc="+str(auc))
        plt.legend(loc=4)
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.show()

In [45]:
# Logisitic Regression 
start_time = datetime.now()

lr = LogisticRegression()
fit_and_evaluate(lr)

end_time = datetime.now()
print('\nDuration: {}'.format(end_time - start_time))

ROC AUC score: 0.4467
Accuracy score: 0.4700
[[13 28]
 [25 34]]
              precision    recall  f1-score   support

           0       0.34      0.32      0.33        41
           1       0.55      0.58      0.56        59

   micro avg       0.47      0.47      0.47       100
   macro avg       0.45      0.45      0.45       100
weighted avg       0.46      0.47      0.47       100



IndexError: Too many indices for array

In [46]:
# Random Forest Classifer
start_time = datetime.now()

rfc = RandomForestClassifier()
fit_and_evaluate(rfc)

end_time = datetime.now()
print('\nDuration: {}'.format(end_time - start_time))

ROC AUC score: 0.4802
Accuracy score: 0.4700
[[22 19]
 [34 25]]
              precision    recall  f1-score   support

           0       0.39      0.54      0.45        41
           1       0.57      0.42      0.49        59

   micro avg       0.47      0.47      0.47       100
   macro avg       0.48      0.48      0.47       100
weighted avg       0.50      0.47      0.47       100



IndexError: Too many indices for array

In [47]:
# Support Vector Classifer
start_time = datetime.now()

smv = SVC()
fit_and_evaluate(smv)

end_time = datetime.now()
print('\nDuration: {}'.format(end_time - start_time))

ROC AUC score: 0.4911
Accuracy score: 0.5400
[[ 9 32]
 [14 45]]
              precision    recall  f1-score   support

           0       0.39      0.22      0.28        41
           1       0.58      0.76      0.66        59

   micro avg       0.54      0.54      0.54       100
   macro avg       0.49      0.49      0.47       100
weighted avg       0.51      0.54      0.51       100



IndexError: Too many indices for array

In [48]:
# KNN Classifer 
start_time = datetime.now()

knn = KNeighborsClassifier()
fit_and_evaluate(knn)

end_time = datetime.now()
print('\nDuration: {}'.format(end_time - start_time))

ROC AUC score: 0.5358
Accuracy score: 0.5400
[[21 20]
 [26 33]]
              precision    recall  f1-score   support

           0       0.45      0.51      0.48        41
           1       0.62      0.56      0.59        59

   micro avg       0.54      0.54      0.54       100
   macro avg       0.53      0.54      0.53       100
weighted avg       0.55      0.54      0.54       100



IndexError: Too many indices for array

In [49]:
# Extreme Gradient Boosting Classifer 
start_time = datetime.now()

xgb = XGBClassifier()
fit_and_evaluate(xgb)

end_time = datetime.now()
print('\nDuration: {}'.format(end_time - start_time))

ValueError: No clients found
Start an client and point it to the scheduler address
  from distributed import Client
  client = Client('ip-addr-of-scheduler:8786')


In [ ]:
# Dataframe to hold the results
model_comparison = pd.DataFrame({'Model': ['Logistic Regression', 'RandomForest Classifier',
                                           'Support Vector Classifer', 'KNeighbors Classifier', 'XGB Classifier'],
                                 'ROC_AUC': [lr_score, rfc_score, svc_score, knn_score, xgb_score]})

# Horizontal bar chart of test mae
model_comparison.sort_values('ROC_AUC', ascending = True)

# Conclusion 
We followed the general outline of a machine learning project:

1. Understand the problem and the data
2. Data cleaning and formatting (this was mostly done for us)
3. Exploratory Data Analysis
4. Baseline model
5. Improved model
6. Model interpretation (just a little)